This notebook extract the list of players who make themselves eligible for the draft.  
The data is extracted from basketball.realgm.com.  
I did not extract player specific details since I will be getting those from thir college profile

In [235]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from lxml import etree, html

In [236]:
years = [2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
link = ['https://basketball.realgm.com/nba/draft/early_entry/by_year/']*len(years)
links = [''.join(item) for item in zip(link,map(str,years))]

In [237]:
# function to extract player names for given year
def draft_player_list(html_tables,year):
    player_name = []
    table_num = []
    year_draft = []
    for tbl_num,table in enumerate(html_tables):
        for item in html.fromstring(html.tostring(table)).xpath('//tbody/tr/td[1]/a'):
            player_name.append(item.text)
            table_num.append(tbl_num)
            year_draft.append(year)
    return pd.DataFrame({'year':year_draft,'table_number':table_num,'player_name':player_name})

In [238]:
# df that assigns the player status, withdrawn from draft or remain in draft for both US and Int players
table_name_lookup = pd.DataFrame({'status':['us_withdrawn','int_withdrawn','us_remain','int_remain'],'table_number':[0,1,2,3]})

# create empty DF to store results
draft_entrantee = pd.DataFrame(columns = ['year','table_number','player_name','status'])
for link in links:
    html_text = requests.get(link)
    # get year from last 4 charcters from link
    year = link[len(link)-4:len(link)]
    
    # get all content from page
    tree = html.fromstring(html_text.content)

    # get all tables on the page
    all_tables = tree.xpath('//body/div[2]/div[3]/div/table')
    
    # get DF with player names
    df = draft_player_list(all_tables,year)
    df = df.merge(table_name_lookup,how = 'left',left_on='table_number',right_on='table_number')
    # append df to main df
    draft_entrantee = pd.concat([draft_entrantee,df],axis = 0)
    

In [239]:
draft_entrantee.to_csv('draft_entrantee.csv',index = False)

In [240]:
# getting other details
# The following is a quick map of the details. But I will not be getting them at this time because I will be focusing on College players only and I will get this data from their college profile.
# td[1]/a - name
# td[2] - position
# td[3] - height
# td[4] - weight
# td[5]/a - dob
# html.fromstring(html.tostring(all_tables[3])).xpath('//tbody/tr/td[5]/a')[0].text

In [247]:
draft_entrantee[draft_entrantee.year.isin(['2008','2020'])].groupby(['year','status']).count()

table_number  player_name
year status                                  
2008 int_remain                5            5
     int_withdrawn            17           17
     us_remain                39           39
     us_withdrawn             31           31
2020 int_remain               13           13
     int_withdrawn            30           30
     us_remain                71           71
     us_withdrawn             94           94

In [ ]:
underclassman - year 1 and 2
upperclassman - year 3 and 4

In [250]:
from sportsipy.ncaab.teams import Team

purdue = Team('PURDUE')

In [262]:
x = purdue.roster

In [263]:
x.players

[Jaden Ivey (jaden-ivey-1),
 Zach Edey (zach-edey-1),
 Trevion Williams (trevion-williams-1),
 Sasha Stefanovic (sasha-stefanovic-1),
 Eric Hunter Jr. (eric-hunterjr-1),
 Mason Gillis (mason-gillis-1),
 Isaiah Thompson (isaiah-thompson-1),
 Caleb Furst (caleb-furst-1),
 Brandon Newman (brandon-newman-2),
 Ethan Morton (ethan-morton-1),
 Carson Barrett (carson-barrett-1),
 Matt Frost (matt-frost-1),
 Chase Martin (chase-martin-1),
 Jared Wulbrun (jared-wulbrun-1),
 Trey Kaufman-Renn (trey-kaufman-renn-1),
 Brian Waddell (brian-waddell-1)]